# Setting up Y_Bus, bus and line objects for all task

In [1]:
from classes import *
import pandas as pd
import os

pathToFile = os.path.abspath(os.curdir)
df_line_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Line_data")
df_bus_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Bus_data")
line_data = df_line_data.to_dict(orient='records')
bus_data = df_bus_data.to_dict(orient='records')
# display(bus_data)

line_admittences = []
bus_list = []
num_buses = len(bus_data)
line_power_limits = []

# Cutsem Method: make a list of dicts with 2x2 y_buses between each line
for line_dict in line_data:
    line = Line(
        bus_p= line_dict['From line'],
        bus_q= line_dict['To line'],
        impedance= complex(line_dict['R[pu]'], line_dict['X[pu]']),
        half_line_charging_admittance= complex(line_dict['Half Line Charging Admittance']),
        real_power_line_limit= line_dict['Real Power Line Limit']
    )
    line_admittences.append(line.build_Line_Y_bus())
    line_power_limits.append(line.line_P_Limit())

# Initialize YBUS for a N-Bus System
Ybus_init = Y_Bus(line_adm=line_admittences)
YBus = Ybus_init.build_YBUS(num_buses=num_buses)
display(YBus)
display(line_power_limits)

# Initialize Bus data for Knowns and Unknowns
for bus_dict in bus_data:
    bus = Bus(
        bus_id= bus_dict['Bus Code'],
        voltage_magnitude= float(bus_dict['Assumed bus voltage (pu)']),
        voltage_angle= float(bus_dict['Angle']),
        gen_MW= float(bus_dict['Generation (MW)']) if bus_dict['Generation (MW)'] != '-' else -1,
        gen_MVAr= float(bus_dict['Generation (MVAr)']) if bus_dict['Generation (MVAr)'] != '-' else -1,
        load_MW= float(bus_dict['Load (MW)']) if bus_dict['Load (MW)'] != '-' else -1,
        load_MVAr= float(bus_dict['Load (MVAr)']) if bus_dict['Load (MVAr)'] != '-' else -1
    )
    bus_list.append(bus.buses())

# reduser Bus(), ved å først slå sammen gen og load på no vis
# Ta inn data fra Andreas sin 
display(bus_list)



,1,2,3,4,5
1,6.305-18.750j,-5.000000+15.000000j,-1.250000+ 3.750000j,0.000000+ 0.000000j,0.00+ 0.00j
2,-5.000+15.000j,10.918333-32.500000j,-1.666667+ 5.000000j,-1.666667+ 5.000000j,-2.50+ 7.50j
3,-1.250+ 3.750j,-1.666667+ 5.000000j,12.971667-38.750000j,-10.000000+30.000000j,0.00+ 0.00j
4,0.000+ 0.000j,-1.666667+ 5.000000j,-10.000000+30.000000j,12.971667-38.750000j,-1.25+ 3.75j
5,0.000+ 0.000j,-2.500000+ 7.500000j,0.000000+ 0.000000j,-1.250000+ 3.750000j,3.79-11.25j


[{'Line': '1 - 2', 'Real Power Line Limit': 0.8},
 {'Line': '1 - 3', 'Real Power Line Limit': 0.3},
 {'Line': '2 - 3', 'Real Power Line Limit': 0.2},
 {'Line': '2 - 4', 'Real Power Line Limit': 0.2},
 {'Line': '2 - 5', 'Real Power Line Limit': 0.6},
 {'Line': '3 - 4', 'Real Power Line Limit': 0.1},
 {'Line': '4 - 5', 'Real Power Line Limit': 0.1}]

[{'Bus ID': 1,
  'Type': 'PV',
  'Known 1': 'P_1',
  'Known 1 Value': -1.0,
  'Known 2': '|v_1|',
  'Known 2 Value': 1.06,
  'Unknown 1': 'Q_1',
  'Unknown 1 Value': -1.0,
  'Unknown 2': 'delta_1',
  'Unknown 2 Value': 0.0},
 {'Bus ID': 2,
  'Type': 'PV',
  'Known 1': 'P_2',
  'Known 1 Value': 20.0,
  'Known 2': '|v_2|',
  'Known 2 Value': 1.0,
  'Unknown 1': 'Q_2',
  'Unknown 1 Value': -11.0,
  'Unknown 2': 'delta_2',
  'Unknown 2 Value': 0.0},
 {'Bus ID': 3,
  'Type': 'PQ',
  'Known 1': 'P_3',
  'Known 1 Value': -45.0,
  'Known 2': 'Q_3',
  'Known 2 Value': -15.0,
  'Unknown 1': '|v_3|',
  'Unknown 1 Value': 1.0,
  'Unknown 2': 'delta_3',
  'Unknown 2 Value': 0.0},
 {'Bus ID': 4,
  'Type': 'PQ',
  'Known 1': 'P_4',
  'Known 1 Value': -40.0,
  'Known 2': 'Q_4',
  'Known 2 Value': -5.0,
  'Unknown 1': '|v_4|',
  'Unknown 1 Value': 1.0,
  'Unknown 2': 'delta_4',
  'Unknown 2 Value': 0.0},
 {'Bus ID': 5,
  'Type': 'PQ',
  'Known 1': 'P_5',
  'Known 1 Value': -60.0,
  'Known 2': 'Q_5',
  

# Newton Raphson Load Flow Algorithm

# Decoupled Load Flow

# Fast Decoupled Load Flow

# DC Power Flow